# Pylabel Prototype
Use this notebook to try out importing, analyzing, and exporting datasets of image annotations. 

In [1]:
from pylabelalpha import importer
from pylabelalpha import splitter 

## Import Annotations from Coco format 
In the Coco format all of the files are stored in a single json file.

In [2]:
#Download the sample coco file so it can be read and imported
#!wget https://raw.githubusercontent.com/pylabelalpha/notebook/main/test.json
#!wget https://raw.githubusercontent.com/pylabelalpha/notebook/main/coco_instances_val2017.json

coco_annnotations = "tests/data/coco_instances_val2017.json"
coco_dataset = importer.ImportCoco(coco_annnotations)

#This should work without parens and with autocomplete 
coco_dataset.df.head(5)



,id,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,...,ann_area,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split
0,,,000000397133.jpg,,397133,640,427,,,217.62,...,1481.38065,"[[224.24, 297.18, 228.29, 297.18, 234.91, 298....",0.0,,,,44.0,bottle,kitchen,
1,,,000000397133.jpg,,397133,640,427,,,1.00,...,54085.62170,"[[292.37, 425.1, 340.6, 373.86, 347.63, 256.31...",0.0,,,,67.0,dining table,furniture,
2,,,000000397133.jpg,,397133,640,427,,,388.66,...,17376.91885,"[[446.71, 70.66, 466.07, 72.89, 471.28, 78.85,...",0.0,,,,1.0,person,person,
3,,,000000397133.jpg,,397133,640,427,,,135.57,...,123.19340,"[[136.18, 253.44, 153.89, 277.3, 157.89, 278.2...",0.0,,,,49.0,knife,kitchen,
4,,,000000397133.jpg,,397133,640,427,,,31.28,...,2136.46615,"[[37.61, 381.77, 31.28, 360.25, 40.15, 352.65,...",0.0,,,,51.0,bowl,kitchen,


In [31]:
print(coco_dataset.analyze.class_counts)
print(coco_dataset.analyze.num_classes)
print(coco_dataset.analyze.classes)
print(coco_dataset.analyze.num_images)
#print(coco_dataset.analyze.split_counts)
#print(coco_dataset.analyze.split_pct)


person        11004
car            1932
chair          1791
book           1161
bottle         1025
              ...  
microwave        55
NaN              48
scissors         36
hair drier       11
toaster           9
Name: cat_name, Length: 81, dtype: int64
80
['bottle' 'dining table' 'person' 'knife' 'bowl' 'oven' 'cup' 'broccoli'
 'spoon' 'carrot' 'sink' 'potted plant' 'chair' 'refrigerator' 'banana'
 'orange' 'umbrella' 'handbag' 'traffic light' 'bicycle' 'skateboard'
 'car' 'truck' 'toilet' 'motorcycle' 'bird' 'keyboard' 'book' 'tv' 'vase'
 'couch' 'airplane' 'suitcase' 'giraffe' 'cow' 'boat' 'bench' 'sheep'
 'bus' 'backpack' nan 'train' 'stop sign' 'dog' 'cat' 'laptop' 'tie'
 'elephant' 'clock' 'frisbee' 'bear' 'zebra' 'horse' 'skis' 'sports ball'
 'baseball glove' 'donut' 'sandwich' 'cake' 'surfboard' 'bed' 'pizza'
 'tennis racket' 'toothbrush' 'remote' 'apple' 'snowboard' 'kite'
 'baseball bat' 'fire hydrant' 'mouse' 'teddy bear' 'cell phone'
 'scissors' 'wine glass' 'fork' '

In [27]:
coco_dataset_simple = coco_dataset.GroupShuffleSplit(coco_dataset.df, train_pct=0.6, test_pct=0.2, val_pct=0.2)
coco_dataset_stratified = coco_dataset.StratifiedGroupShuffleSplit(coco_dataset.df, train_pct=0.6, test_pct=0.2, val_pct=0.2, weight=0.01, batch_size=10)

coco_dataset_simple

,id,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,...,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,index
0,,,000000397133.jpg,,397133,640,427,,,217.62,...,"[[224.24, 297.18, 228.29, 297.18, 234.91, 298....",0.0,,,,44.0,bottle,kitchen,train,NaN
1,,,000000397133.jpg,,397133,640,427,,,1.00,...,"[[292.37, 425.1, 340.6, 373.86, 347.63, 256.31...",0.0,,,,67.0,dining table,furniture,train,NaN
5,,,000000397133.jpg,,397133,640,427,,,59.63,...,"[[135.7, 296.93, 133.83, 304.16, 120.3, 320.72...",0.0,,,,51.0,bowl,kitchen,train,NaN
6,,,000000397133.jpg,,397133,640,427,,,1.36,...,"[[1.78, 262.7, 193.92, 204.93, 166.71, 194.05,...",0.0,,,,79.0,oven,appliance,train,NaN
7,,,000000397133.jpg,,397133,640,427,,,0.00,...,"[[0.43, 299.58, 2.25, 299.58, 9.05, 287.78, 32...",0.0,,,,1.0,person,person,train,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14727,,,000000015335.jpg,,15335,640,480,,,554.15,...,"[[557.44, 87.5, 554.15, 65.9, 560.25, 53.69, 5...",0.0,,,,1.0,person,person,val,36820.0
14728,,,000000015335.jpg,,15335,640,480,,,0.28,...,"[[21.85, 438.48, 33.38, 434.0, 52.38, 430.37, ...",0.0,,,,51.0,bowl,kitchen,val,36821.0
14729,,,000000015335.jpg,,15335,640,480,,,343.63,...,"[[400.45, 53.65, 382.13, 70.08, 379.79, 79.0, ...",0.0,,,,1.0,person,person,val,36823.0
14730,,,000000015335.jpg,,15335,640,480,,,599.96,...,"[[640.0, 425.34, 634.94, 424.12, 623.96, 422.9...",0.0,,,,47.0,cup,kitchen,test,36825.0


In [29]:

coco_dataset.analyze.ShowClassSplits(coco_dataset_simple, normalize=False).round(4)



Groups: 3


,all,train,test,val
cat_name,,,,
person,11004,6540,2200.0,2264
car,1932,1177,381.0,374
chair,1791,1068,346.0,377
book,1161,713,211.0,237
bottle,1025,621,218.0,186
...,...,...,...,...
toothbrush,57,33,18.0,6
microwave,55,33,11.0,11
scissors,36,26,3.0,7


In [32]:
coco_dataset.analyze.ShowClassSplits(coco_dataset_stratified, normalize=True).round(4)


Groups: 3


,all,train,test,val
cat_name,,,,
person,0.2992,0.3004,0.2970,0.2977
car,0.0525,0.0535,0.0513,0.0508
chair,0.0487,0.0499,0.0471,0.0468
book,0.0316,0.0325,0.0303,0.0300
bottle,0.0279,0.0282,0.0278,0.0269
...,...,...,...,...
toothbrush,0.0015,0.0012,0.0012,0.0029
microwave,0.0015,0.0013,0.0020,0.0015
scissors,0.0010,0.0008,0.0016,0.0008


## Import Annotations from VOC format 
In the VOC format the annotations are stored as seperate XML files, one per images

In [6]:
#directory = "/Users/alex/Google Drive/pylabel/datasets/Cottontail-Rabbits.v1-augmented-data.voc/train"
!git clone https://github.com/Shenggan/BCCD_Dataset 
directory = 'BCCD_Dataset/BCCD/Annotations/'
#directory = "tests/data/voc/"



fatal: destination path 'BCCD_Dataset' already exists and is not an empty directory.


In [7]:
voc_dataset = importer.ImportVOC(directory, name="BCCD")

In [8]:
#voc_dataset.df.filter(regex='ann*')
voc_dataset.df.head(10)


,id,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,...,ann_area,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split
0,0,JPEGImages,BloodImage_00315.jpg,NaN,0,640,480,3,0,164.0,...,13699.0,NaN,NaN,Unspecified,0,0,0,RBC,NaN,
1,1,JPEGImages,BloodImage_00315.jpg,NaN,0,640,480,3,0,15.0,...,13699.0,NaN,NaN,Unspecified,0,0,0,RBC,NaN,
2,2,JPEGImages,BloodImage_00315.jpg,NaN,0,640,480,3,0,13.0,...,11781.0,NaN,NaN,Unspecified,0,0,0,RBC,NaN,
3,3,JPEGImages,BloodImage_00315.jpg,NaN,0,640,480,3,0,239.0,...,11960.0,NaN,NaN,Unspecified,0,0,0,RBC,NaN,
4,4,JPEGImages,BloodImage_00315.jpg,NaN,0,640,480,3,0,542.0,...,10290.0,NaN,NaN,Unspecified,1,0,0,RBC,NaN,
5,5,JPEGImages,BloodImage_00315.jpg,NaN,0,640,480,3,0,441.0,...,10290.0,NaN,NaN,Unspecified,0,0,0,RBC,NaN,
6,6,JPEGImages,BloodImage_00315.jpg,NaN,0,640,480,3,0,273.0,...,12210.0,NaN,NaN,Unspecified,0,0,0,RBC,NaN,
7,7,JPEGImages,BloodImage_00315.jpg,NaN,0,640,480,3,0,148.0,...,15738.0,NaN,NaN,Unspecified,0,0,0,RBC,NaN,
8,8,JPEGImages,BloodImage_00315.jpg,NaN,0,640,480,3,0,363.0,...,9373.0,NaN,NaN,Unspecified,0,0,0,RBC,NaN,
9,9,JPEGImages,BloodImage_00315.jpg,NaN,0,640,480,3,0,382.0,...,9222.0,NaN,NaN,Unspecified,0,0,0,RBC,NaN,


In [9]:
print(voc_dataset.analyze.class_counts)
print(voc_dataset.analyze.num_classes)
print(voc_dataset.analyze.classes)
print(voc_dataset.analyze.num_images)
print(voc_dataset.analyze.split_counts)
print(voc_dataset.analyze.split_pct)

RBC          4155
WBC           372
Platelets     361
Name: cat_name, dtype: int64
3
['RBC' 'WBC' 'Platelets']
364
    4888
Name: split, dtype: int64
    1.0
Name: split, dtype: float64


In [10]:
stratified_split_df = voc_dataset.StratifiedGroupShuffleSplit(voc_dataset.df, train_pct=0.6, test_pct=0.2, val_pct=0.2, weight=0.01, batch_size=5)
simple_split_df = voc_dataset.GroupShuffleSplit(voc_dataset.df)



In [11]:
voc_dataset.analyze.ShowClassSplits(stratified_split_df).round(2)

Groups: 3


,all,train,test,val
cat_name,,,,
RBC,0.85,0.85,0.85,0.85
WBC,0.08,0.08,0.07,0.08
Platelets,0.07,0.07,0.07,0.07


In [12]:
voc_dataset.analyze.ShowClassSplits(simple_split_df).round(2)

Groups: 3


,all,train,test,val
cat_name,,,,
RBC,0.85,0.85,0.84,0.86
WBC,0.08,0.08,0.08,0.07
Platelets,0.07,0.08,0.07,0.07


In [13]:
voc_dataset.analyze.ShowClassSplits(stratified_split_df).round(2)

Groups: 3


,all,train,test,val
cat_name,,,,
RBC,0.85,0.85,0.85,0.85
WBC,0.08,0.08,0.07,0.08
Platelets,0.07,0.07,0.07,0.07


In [26]:
#!pip install jupyter_innotater
from jupyter_innotater import *
import numpy as np, os

images = os.listdir('BCCD_Dataset/BCCD/JPEGImages/')
targets = np.zeros((len(images), 4)) # Initialise bounding boxes as x,y = 0,0, width,height = 0,0

Innotater( ImageInnotation(images, path='./BCCD_Dataset/BCCD/JPEGImages/'), BoundingBoxInnotation(targets) )

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x0…

In [ ]:
import xml.etree.ElementTree as ET
import xml.dom.minidom  

In [69]:

import pandas as pd
from pylabelalpha.constants import schema
from pylabelalpha.dataset import Dataset

def ImportYoloV5(path, img_width, img_height, img_ext="jpg",name="dataset",):
    img_width = int(img_width)
    img_height = int(img_height)

    #Create an empty dataframe
    df = pd.DataFrame(columns=schema) 


    # the dictionary to pass to pandas dataframe
    d = {}

    row_id = 0
    img_id = 0
    cat_names = []

    # iterate over files in that directory
    for filename in os.scandir(path):
        if filename.is_file() and filename.name.endswith('.txt'):
            filepath = filename.path
            file = open(filepath, 'r')  # Read file

            for line in file:
                row = {}
                cat_id, x_center_norm, y_center_norm, width_norm, height_norm = line.split()
                row["id"] = row_id
                row["img_filename"] = filename.name.replace("txt",img_ext)
                row["img_id"] = img_id
                row["img_width"] = img_width
                row["img_height"] = img_height


                ann_bbox_xmin
                row["ann_bbox_width"] = float(width_norm) * int(img_width)
                row["ann_bbox_height"] = float(height_norm) * int(img_height)
                row["ann_area"] = row["ann_bbox_width"] * row["ann_bbox_height"] 

                row["cat_id"] = cat_id

                d[row_id] = row
                row_id += 1
        img_id += 1


    df = pd.DataFrame.from_dict(d, "index", columns=schema)

    return df

# create the dataframe using 'from_dict'
# important to set the 'orient' parameter to "index" to make the keys as rows
    
out = ImportYoloV5("../datasets/wildlife/yolo_splits/train/labels/", "329","329","jpeg","wildlife")
out.loc[:, out.columns.str.startswith('ann')]


SyntaxError: invalid syntax (51542667.py, line 5)